In [6]:
# モジュールのimport
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.dates as mdate

pd.set_option('display.max_columns', None) # pandasオプション：列データを全て表示

# データフレームdfにOur World in Dataのデータを読み込む
df = pd.read_json("owid-covid-data.json")
df = df[df.columns.drop(list(df.filter(regex='OWID')))] # OWIDデータを除外

df

,AFG,ALB,DZA,ASM,AND,AGO,AIA,ATG,ARG,ARM,ABW,AUS,AUT,AZE,BHS,BHR,BGD,BRB,BLR,BEL,BLZ,BEN,BMU,BTN,BOL,BES,BIH,BWA,BRA,VGB,BRN,BGR,BFA,BDI,KHM,CMR,CAN,CPV,CYM,CAF,TCD,CHL,CHN,COL,COM,COG,COK,CRI,CIV,HRV,CUB,CUW,CYP,CZE,COD,DNK,DJI,DMA,DOM,ECU,EGY,SLV,GNQ,ERI,EST,SWZ,ETH,FRO,FLK,FJI,FIN,FRA,GUF,PYF,GAB,GMB,GEO,DEU,GHA,GIB,GRC,GRL,GRD,GLP,GUM,GTM,GGY,GIN,GNB,GUY,HTI,HND,HKG,HUN,ISL,IND,IDN,IRN,IRQ,IRL,IMN,ISR,ITA,JAM,JPN,JEY,JOR,KAZ,KEN,KIR,KWT,KGZ,LAO,LVA,LBN,LSO,LBR,LBY,LIE,LTU,LUX,MAC,MDG,MWI,MYS,MDV,MLI,MLT,MHL,MTQ,MRT,MUS,MYT,MEX,FSM,MDA,MCO,MNG,MNE,MSR,MAR,MOZ,MMR,NAM,NRU,NPL,NLD,NCL,NZL,NIC,NER,NGA,NIU,PRK,MKD,MNP,NOR,OMN,PAK,PLW,PSE,PAN,PNG,PRY,PER,PHL,PCN,POL,PRT,PRI,QAT,REU,ROU,RUS,RWA,BLM,SHN,KNA,LCA,MAF,SPM,VCT,WSM,SMR,STP,SAU,SEN,SRB,SYC,SLE,SGP,SXM,SVK,SVN,SLB,SOM,ZAF,KOR,SSD,ESP,LKA,SDN,SUR,SWE,CHE,SYR,TWN,TJK,TZA,THA,TLS,TGO,TKL,TON,TTO,TUN,TUR,TKM,TCA,TUV,UGA,UKR,ARE,GBR,USA,VIR,URY,UZB,VUT,VAT,VEN,VNM,WLF,ESH,YEM,ZMB,ZWE
continent,Asia,Europe,Africa,Oceania,Europe,Africa,North America,North America,South America,Asia,North America,Oceania,Europe,Asia,North America,Asia,Asia,North America,Europe,Europe,North America,Africa,North America,Asia,South America,North America,Europe,Africa,South America,North America,Asia,Europe,Africa,Africa,Asia,Africa,North America,Africa,North America,Africa,Africa,South America,Asia,South America,Africa,Africa,Oceania,North America,Africa,Europe,North America,North America,Europe,Europe,Africa,Europe,Africa,North America,North America,South America,Africa,North America,Africa,Africa,Europe,Africa,Africa,Europe,South America,Oceania,Europe,Europe,South America,Oceania,Africa,Africa,Asia,Europe,Africa,Europe,Europe,North America,North America,North America,Oceania,North America,Europe,Africa,Africa,South America,North America,North America,Asia,Europe,Europe,Asia,Asia,Asia,Asia,Europe,Europe,Asia,Europe,North America,Asia,Europe,Asia,Asia,Africa,Oceania,Asia,Asia,Asia,Europe,Asia,Africa,Africa,Africa,Europe,Europe,Europe,Asia,Africa,Africa,Asia,Asia,Africa,Europe,Oceania,North America,Africa,Africa,Africa,North America,Oceania,Europe,Europe,Asia,Europe,North America,Africa,Africa,Asia,Africa,Oceania,Asia,Europe,Oceania,Oceania,North America,Africa,Africa,Oceania,Asia,Europe,Oceania,Europe,Asia,Asia,Oceania,Asia,North America,Oceania,South America,South America,Asia,Oceania,Europe,Europe,North America,Asia,Africa,Europe,Europe,Africa,North America,Africa,North America,North America,North America,North America,North America,Oceania,Europe,Africa,Asia,Africa,Europe,Africa,Africa,Asia,North America,Europe,Europe,Oceania,Africa,Africa,Asia,Africa,Europe,Asia,Africa,South America,Europe,Europe,Asia,Asia,Asia,Africa,Asia,Asia,Africa,Oceania,Oceania,North America,Africa,Asia,Asia,North America,Oceania,Africa,Europe,Asia,Europe,North America,North America,South America,Asia,Oceania,Europe,South America,Asia,Oceania,Africa,Asia,Africa,Africa
location,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,Aruba,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bermuda,Bhutan,Bolivia,Bonaire Sint Eustatius and Saba,Bosnia and Herzegovina,Botswana,Brazil,British Virgin Islands,Brunei,Bulgaria,Burkina Faso,Burundi,Cambodia,Cameroon,Canada,Cape Verde,Cayman Islands,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo,Cook Islands,Costa Rica,Cote d'Ivoire,Croatia,Cuba,Curacao,Cyprus,Czechia,Democratic Republic of Congo,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Equatorial Guinea,Eritrea,Estonia,Eswatini,Ethiopia,Faeroe Islands,Falkland Islands,Fiji,Finland,France,French Guiana,French Polynesia,Gabon,Gambia,Georgia,Germany,Ghana,Gibraltar,Greece,Greenland,Grenada,Guadeloupe,Guam,Guatemala,Guernsey,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,Hong Kong,Hungary,Iceland,India,Indonesia,Iran,Iraq,Ireland,Isle of Man,Israel,Italy,Jamaica,Japan,Jersey,Jordan,Kazakhstan,Kenya,Ki

In [7]:
countries = df.columns # 列の国名コード
diff_day = ('max_day', 'max_correl')

df2 = pd.DataFrame(columns = countries, index = diff_day) # index：感染情報，column：国名コードとなるdf2を作成する

# 型変換：objectをfloatへと変換
dft  = df.T.apply(pd.to_numeric, errors='ignore')
dft2 = df2.T.apply(pd.to_numeric, errors='coerce')

# 二つのデータフレームをマージ
df_all = pd.concat([dft,dft2], axis=1)
df_all = df_all.T

df_all

,AFG,ALB,DZA,ASM,AND,AGO,AIA,ATG,ARG,ARM,ABW,AUS,AUT,AZE,BHS,BHR,BGD,BRB,BLR,BEL,BLZ,BEN,BMU,BTN,BOL,BES,BIH,BWA,BRA,VGB,BRN,BGR,BFA,BDI,KHM,CMR,CAN,CPV,CYM,CAF,TCD,CHL,CHN,COL,COM,COG,COK,CRI,CIV,HRV,CUB,CUW,CYP,CZE,COD,DNK,DJI,DMA,DOM,ECU,EGY,SLV,GNQ,ERI,EST,SWZ,ETH,FRO,FLK,FJI,FIN,FRA,GUF,PYF,GAB,GMB,GEO,DEU,GHA,GIB,GRC,GRL,GRD,GLP,GUM,GTM,GGY,GIN,GNB,GUY,HTI,HND,HKG,HUN,ISL,IND,IDN,IRN,IRQ,IRL,IMN,ISR,ITA,JAM,JPN,JEY,JOR,KAZ,KEN,KIR,KWT,KGZ,LAO,LVA,LBN,LSO,LBR,LBY,LIE,LTU,LUX,MAC,MDG,MWI,MYS,MDV,MLI,MLT,MHL,MTQ,MRT,MUS,MYT,MEX,FSM,MDA,MCO,MNG,MNE,MSR,MAR,MOZ,MMR,NAM,NRU,NPL,NLD,NCL,NZL,NIC,NER,NGA,NIU,PRK,MKD,MNP,NOR,OMN,PAK,PLW,PSE,PAN,PNG,PRY,PER,PHL,PCN,POL,PRT,PRI,QAT,REU,ROU,RUS,RWA,BLM,SHN,KNA,LCA,MAF,SPM,VCT,WSM,SMR,STP,SAU,SEN,SRB,SYC,SLE,SGP,SXM,SVK,SVN,SLB,SOM,ZAF,KOR,SSD,ESP,LKA,SDN,SUR,SWE,CHE,SYR,TWN,TJK,TZA,THA,TLS,TGO,TKL,TON,TTO,TUN,TUR,TKM,TCA,TUV,UGA,UKR,ARE,GBR,USA,VIR,URY,UZB,VUT,VAT,VEN,VNM,WLF,ESH,YEM,ZMB,ZWE
continent,Asia,Europe,Africa,Oceania,Europe,Africa,North America,North America,South America,Asia,North America,Oceania,Europe,Asia,North America,Asia,Asia,North America,Europe,Europe,North America,Africa,North America,Asia,South America,North America,Europe,Africa,South America,North America,Asia,Europe,Africa,Africa,Asia,Africa,North America,Africa,North America,Africa,Africa,South America,Asia,South America,Africa,Africa,Oceania,North America,Africa,Europe,North America,North America,Europe,Europe,Africa,Europe,Africa,North America,North America,South America,Africa,North America,Africa,Africa,Europe,Africa,Africa,Europe,South America,Oceania,Europe,Europe,South America,Oceania,Africa,Africa,Asia,Europe,Africa,Europe,Europe,North America,North America,North America,Oceania,North America,Europe,Africa,Africa,South America,North America,North America,Asia,Europe,Europe,Asia,Asia,Asia,Asia,Europe,Europe,Asia,Europe,North America,Asia,Europe,Asia,Asia,Africa,Oceania,Asia,Asia,Asia,Europe,Asia,Africa,Africa,Africa,Europe,Europe,Europe,Asia,Africa,Africa,Asia,Asia,Africa,Europe,Oceania,North America,Africa,Africa,Africa,North America,Oceania,Europe,Europe,Asia,Europe,North America,Africa,Africa,Asia,Africa,Oceania,Asia,Europe,Oceania,Oceania,North America,Africa,Africa,Oceania,Asia,Europe,Oceania,Europe,Asia,Asia,Oceania,Asia,North America,Oceania,South America,South America,Asia,Oceania,Europe,Europe,North America,Asia,Africa,Europe,Europe,Africa,North America,Africa,North America,North America,North America,North America,North America,Oceania,Europe,Africa,Asia,Africa,Europe,Africa,Africa,Asia,North America,Europe,Europe,Oceania,Africa,Africa,Asia,Africa,Europe,Asia,Africa,South America,Europe,Europe,Asia,Asia,Asia,Africa,Asia,Asia,Africa,Oceania,Oceania,North America,Africa,Asia,Asia,North America,Oceania,Africa,Europe,Asia,Europe,North America,North America,South America,Asia,Oceania,Europe,South America,Asia,Oceania,Africa,Asia,Africa,Africa
location,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,Aruba,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bermuda,Bhutan,Bolivia,Bonaire Sint Eustatius and Saba,Bosnia and Herzegovina,Botswana,Brazil,British Virgin Islands,Brunei,Bulgaria,Burkina Faso,Burundi,Cambodia,Cameroon,Canada,Cape Verde,Cayman Islands,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo,Cook Islands,Costa Rica,Cote d'Ivoire,Croatia,Cuba,Curacao,Cyprus,Czechia,Democratic Republic of Congo,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Equatorial Guinea,Eritrea,Estonia,Eswatini,Ethiopia,Faeroe Islands,Falkland Islands,Fiji,Finland,France,French Guiana,French Polynesia,Gabon,Gambia,Georgia,Germany,Ghana,Gibraltar,Greece,Greenland,Grenada,Guadeloupe,Guam,Guatemala,Guernsey,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,Hong Kong,Hungary,Iceland,India,Indonesia,Iran,Iraq,Ireland,Isle of Man,Israel,Italy,Jamaica,Japan,Jersey,Jordan,Kazakhstan,Kenya,Ki

In [8]:
def contains_column_name(df, substring):
    # 各列名が特定の文字列を含むかどうかを判定
    result = df.columns.str.contains(substring).any()
    return result

In [9]:
def insert_column (df_all, key):
    def country_search (country_code1):
        country1 = pd.DataFrame(df_all[country_code1]['data'])
        country1['date'] = pd.to_datetime(country1['date']) # 日付フォーマットを一括変換
        country1.set_index('date',inplace = True) # dateをindexに設定
        if contains_column_name(country1, key):
            cdata1 = country1.dropna(subset = [key])
            if not cdata1.empty:
                v1_mean = cdata1[key].mean()
                v1_std  = cdata1[key].std(ddof=0)
                df_all[country_code1]['max_day'] = v1_mean
                df_all[country_code1]['max_correl'] = v1_std
            else:
                df_all[country_code1]['max_day'] = np.nan
                df_all[country_code1]['max_correl'] = np.nan
    return country_search

new_cases_insert = insert_column(df_all,'new_cases_smoothed')

countries = df_all.columns

for country in countries:
    new_cases_insert(country)

df_all

,AFG,ALB,DZA,ASM,AND,AGO,AIA,ATG,ARG,ARM,ABW,AUS,AUT,AZE,BHS,BHR,BGD,BRB,BLR,BEL,BLZ,BEN,BMU,BTN,BOL,BES,BIH,BWA,BRA,VGB,BRN,BGR,BFA,BDI,KHM,CMR,CAN,CPV,CYM,CAF,TCD,CHL,CHN,COL,COM,COG,COK,CRI,CIV,HRV,CUB,CUW,CYP,CZE,COD,DNK,DJI,DMA,DOM,ECU,EGY,SLV,GNQ,ERI,EST,SWZ,ETH,FRO,FLK,FJI,FIN,FRA,GUF,PYF,GAB,GMB,GEO,DEU,GHA,GIB,GRC,GRL,GRD,GLP,GUM,GTM,GGY,GIN,GNB,GUY,HTI,HND,HKG,HUN,ISL,IND,IDN,IRN,IRQ,IRL,IMN,ISR,ITA,JAM,JPN,JEY,JOR,KAZ,KEN,KIR,KWT,KGZ,LAO,LVA,LBN,LSO,LBR,LBY,LIE,LTU,LUX,MAC,MDG,MWI,MYS,MDV,MLI,MLT,MHL,MTQ,MRT,MUS,MYT,MEX,FSM,MDA,MCO,MNG,MNE,MSR,MAR,MOZ,MMR,NAM,NRU,NPL,NLD,NCL,NZL,NIC,NER,NGA,NIU,PRK,MKD,MNP,NOR,OMN,PAK,PLW,PSE,PAN,PNG,PRY,PER,PHL,PCN,POL,PRT,PRI,QAT,REU,ROU,RUS,RWA,BLM,SHN,KNA,LCA,MAF,SPM,VCT,WSM,SMR,STP,SAU,SEN,SRB,SYC,SLE,SGP,SXM,SVK,SVN,SLB,SOM,ZAF,KOR,SSD,ESP,LKA,SDN,SUR,SWE,CHE,SYR,TWN,TJK,TZA,THA,TLS,TGO,TKL,TON,TTO,TUN,TUR,TKM,TCA,TUV,UGA,UKR,ARE,GBR,USA,VIR,URY,UZB,VUT,VAT,VEN,VNM,WLF,ESH,YEM,ZMB,ZWE
continent,Asia,Europe,Africa,Oceania,Europe,Africa,North America,North America,South America,Asia,North America,Oceania,Europe,Asia,North America,Asia,Asia,North America,Europe,Europe,North America,Africa,North America,Asia,South America,North America,Europe,Africa,South America,North America,Asia,Europe,Africa,Africa,Asia,Africa,North America,Africa,North America,Africa,Africa,South America,Asia,South America,Africa,Africa,Oceania,North America,Africa,Europe,North America,North America,Europe,Europe,Africa,Europe,Africa,North America,North America,South America,Africa,North America,Africa,Africa,Europe,Africa,Africa,Europe,South America,Oceania,Europe,Europe,South America,Oceania,Africa,Africa,Asia,Europe,Africa,Europe,Europe,North America,North America,North America,Oceania,North America,Europe,Africa,Africa,South America,North America,North America,Asia,Europe,Europe,Asia,Asia,Asia,Asia,Europe,Europe,Asia,Europe,North America,Asia,Europe,Asia,Asia,Africa,Oceania,Asia,Asia,Asia,Europe,Asia,Africa,Africa,Africa,Europe,Europe,Europe,Asia,Africa,Africa,Asia,Asia,Africa,Europe,Oceania,North America,Africa,Africa,Africa,North America,Oceania,Europe,Europe,Asia,Europe,North America,Africa,Africa,Asia,Africa,Oceania,Asia,Europe,Oceania,Oceania,North America,Africa,Africa,Oceania,Asia,Europe,Oceania,Europe,Asia,Asia,Oceania,Asia,North America,Oceania,South America,South America,Asia,Oceania,Europe,Europe,North America,Asia,Africa,Europe,Europe,Africa,North America,Africa,North America,North America,North America,North America,North America,Oceania,Europe,Africa,Asia,Africa,Europe,Africa,Africa,Asia,North America,Europe,Europe,Oceania,Africa,Africa,Asia,Africa,Europe,Asia,Africa,South America,Europe,Europe,Asia,Asia,Asia,Africa,Asia,Asia,Africa,Oceania,Oceania,North America,Africa,Asia,Asia,North America,Oceania,Africa,Europe,Asia,Europe,North America,North America,South America,Asia,Oceania,Europe,South America,Asia,Oceania,Africa,Asia,Africa,Africa
location,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,Aruba,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bermuda,Bhutan,Bolivia,Bonaire Sint Eustatius and Saba,Bosnia and Herzegovina,Botswana,Brazil,British Virgin Islands,Brunei,Bulgaria,Burkina Faso,Burundi,Cambodia,Cameroon,Canada,Cape Verde,Cayman Islands,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo,Cook Islands,Costa Rica,Cote d'Ivoire,Croatia,Cuba,Curacao,Cyprus,Czechia,Democratic Republic of Congo,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Equatorial Guinea,Eritrea,Estonia,Eswatini,Ethiopia,Faeroe Islands,Falkland Islands,Fiji,Finland,France,French Guiana,French Polynesia,Gabon,Gambia,Georgia,Germany,Ghana,Gibraltar,Greece,Greenland,Grenada,Guadeloupe,Guam,Guatemala,Guernsey,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,Hong Kong,Hungary,Iceland,India,Indonesia,Iran,Iraq,Ireland,Isle of Man,Israel,Italy,Jamaica,Japan,Jersey,Jordan,Kazakhstan,Kenya,Ki

In [ ]:
def crosscorrel (df, key):
    def cross1(country_code1):
        def cross2 (country_code2):
            country1 = pd.DataFrame(df[country_code1]['data'])
            country1['date'] = pd.to_datetime(country1['date']) # 日付フォーマットを一括変換
            country1.set_index('date',inplace = True) # dateをindexに設定

            # 国のデータをcountry2へ格納
            country2 = pd.DataFrame(df[country_code2]['data'])
            country2['date'] = pd.to_datetime(country2['date']) # 日付フォーマットを一括変換
            country2.set_index('date',inplace = True)

            cdata1 = country1.dropna(subset = [key])
            cdata2 = country2.dropna(subset = [key])
            dates1 = cdata1.index
            dates2 = cdata2.index

            # keyの平均値・分散値
            v1_mean = cdata1[key].mean()
            v1_std  = cdata1[key].std(ddof=0)
            v2_mean = cdata2[key].mean()
            v2_std  = cdata2[key].std(ddof=0)

            # 相関の計算に必要なデータの抽出
            numDates1 = len(dates1)
            numDates2 = len(dates2)
            maxDiffDay1 = (dates2[-1] - dates1[0]).days
            maxDiffDay2 = (dates1[-1] - dates2[0]).days
            correlSize = maxDiffDay1 + maxDiffDay2 + 1
            diffs = np.arange(- maxDiffDay2, maxDiffDay1 + 1)
            correlations = np.zeros([correlSize])
            counts = np.zeros([correlSize])

            # 相互相関の計算
            for i_day in range(numDates1):
                for j_day in range(numDates2):

                    # i_dayとj_dayを取り出し日の差をdiffDayに格納
                    # （maxDiffDay - 1）は配列の0スタートのためのオフセット
                    diffDay = (dates2[j_day] - dates1[i_day]).days + maxDiffDay2

                    # 相互相関を計算
                    correlations[diffDay] += (cdata1[key][i_day] - v1_mean)*(cdata2[key][j_day] - v2_mean)

                    # 足した回数を記録
                    counts[diffDay] += 1

            # 相互相関をkey1の標準偏差*key2の標準偏差*足した回数で割る
            correlations /= v1_std*v2_std*counts

            print('===== ', country_code2 , '：計算完了 ===== ')

            diffDayMax = +200
            diffDayMin = -200

            # グラフを表示する範囲から最大値の場所を抜き出す
            diffs = np.array(diffs)
            correlations = np.array(correlations)
            cliped_diffs = diffs[(diffs > diffDayMin) & (diffs < diffDayMax)]
            cliped_correlations = correlations[(diffs > diffDayMin) & (diffs < diffDayMax)]
            max_day    = cliped_diffs[cliped_correlations.argmax()]
            max_correl = cliped_correlations[cliped_correlations.argmax()]

            return max_day, max_correl

        return cross2

    return cross1

In [ ]:
countries = df.columns # 列の国名コード
diff_day = ('max_day', 'max_correl')

df2 = pd.DataFrame(columns = countries, index = diff_day) # index：感染情報，column：国名コードとなるdf2を作成する

# 各国ごとに最新の有効データを取得
for country in countries:
    df_country = pd.DataFrame(df[country]['data'])
    latest_data = CHN_cross(country)

    # 新しいDataFrameにデータを格納
    df2[country] = pd.Series(latest_data)

# 型変換：objectをfloatへと変換
dft  = df.T.apply(pd.to_numeric, errors='ignore')
dft2 = df2.T.apply(pd.to_numeric, errors='coerce')

# 二つのデータフレームをマージ
df_all = pd.concat([dft,dft2], axis=1)

df_all # df_allの表示

In [10]:
df_all.to_csv("correl.csv")